# Kernel Learning(カーネル学習)

Kernel learning is a way to transform features in either classification or regression problems. Recall in regression, we have the following model equation:
(カーネル学習は分類、または回帰問題のどちらかで特徴量を変換する方法です。回帰問題では、以下のような回帰式を用います。)
\begin{equation}
    \hat{y} = \vec{w}\vec{x} + b
\end{equation}

where $\vec{x}$ is our feature vector of dimension $D$. In kernel learning, we transform our feature vector from dimension $D$ features to *distances to training data points* of dimension $N$, where $N$ is the number of training data points:
（ここで、 $\vec{x}$は次元$D$の特徴ベクトルです。カーネル学習では次元$D$の特徴ベクトルから次元$N$の*訓練データ点との距離*に変換します。このときの$N$は訓練データ点の数です。）
\begin{equation}
    \hat{y} = \sum_i^N w_i \left<\vec{x}, \vec{x}_i\right>+ b
\end{equation}

where $\left<\cdot\right>$ is distance between two feature vectors and $\vec{x}_i$ is the $i$th training data point. $\vec{x}$ is the function argument whereas $\vec{x}_i$ are known values. 
(上式の$\left<\cdot\right>$は２つの特徴ベクトル間の距離であり$\vec{x}_i$は$i$番目の訓練データ点です。$\vec{x}_i$は既知の値ですが、$\vec{x}$は関数の引数です。)
```{admonition} Audience & Objectives（この章の対象者と目的）
This chapter builds on {doc}`regression` and {doc}`classification`. After completing this chapter, you should be able to 

  * Distinguish between primal and dual form
  * Choose when kernel learning could be beneficial
  * Understand the principles of training curves and the connection between training size and feature number  
  
  （この章は{doc}`regression`と{doc}`classification`に基づいて進められます。この章を修了すると
  * 主形式と双対形式の違いを見抜けられます。
  * カーネル学習が上手くいきそうだと思ったときにカーネル学習をチョイスできます。
  * 学習曲線の原理や訓練データサイズと特徴量の個数の間の関係を理解することができます。
）
```


```{margin}
Although we will use the word distance, the $\left<\cdot\right>$ function is actually an inner product which is more flexible than a distance.
(距離というワードを使っていきますが、実際には$\left<\cdot\right>$は内積であって距離よりフレキシブルなものになります。)
```

One of the consequences of this transformation is that our training weight vector, $\vec{w}$, no longer depends on the number of features. **Instead the number of weights depends on the number of training data points.** This is the reason we use kernel learning. You might have few training data points but large feature vectors ($N < D$). By using the kernel formulation, you'll reduce the number of weights. It might also be that your feature space is hard to model with a linear equation, but when you view it as distances from training data it becomes linear (often $N > D$). Finally, it could be that your feature vector is infinite dimensional (i.e., it is a function not a vector) or for some reason you cannot compute it. In kernel learning, you only need to define your *kernel function*  $\left<\cdot\right>$ and never explicitly work with the feature vector. 
（この変換による帰結として、訓練データの重みベクトルが特徴量の個数に全く依存しなくなるということが起こります。**その代わりに重みの個数は訓練データ点の個数に依存します。**これがカーネル学習を用いる理由になります。トレーニングデータがほんの少ししかないのに特徴量ベクトルの次元が大きい($N < D$)といったことはよくあるかもしれません。カーネル定式化を使うことで重みの個数を減らすことができます。特徴空間が線形方定式でモデリングすることが困難な時でも、それを訓練データとの距離とみなすことで線形として扱えます(多くの場合、$N > D$)。最後に、特徴ベクトルが無限次元(i.e. ベクトルではなく関数のとき)となったり、何らかの理由で特徴ベクトルを計算できないときもあったりします。カーネル学習では、*カーネル関数*$\left<\cdot\right>$を定義する必要があるだけで特徴ベクトルを明示的に扱う必要はありません。）

The distance function is called a *kernel function*  $\left<\cdot\right>$. A kernel function is a binary function (takes two arguments) that outputs a scalar and has the following properties:
（距離の関数は*カーネル関数*$\left<\cdot\right>$と呼ばれています。カーネル関数は二変数関数（２つの引数を受け取る）であってスカラー値を出力し、以下のような性質を持ちます。）
1. Positive: $\left<x, x'\right> \geq 0$
2. Symmetric: $\left<x, x'\right> = \left<x', x \right>$
3. Point-separating: $\left<x, x'\right> = 0$ if and only if $x = x'$
(
1. 正定性(Positive): $\left<x, x'\right> \geq 0$
2. 対称性(Symmetric): $\left<x, x'\right> = \left<x', x \right>$
3. Point-seperating: $\left<x, x'\right> = 0$ if and only if $x = x'$

訳者注：Point-separatingにふさわしい訳語が無かったため原著の英語のみを記載した。3番目の性質に類似したものとして距離関数の定義の1つの非退化性がある。
)
The classic kernel function example is $l_2$ norm (Euclidean distance): $\left<\vec{x}, \vec{x}'\right>=\sqrt{\sum^D_i (x_i - x_i^{'})^2}$. Some of the most interesting applications of kernel learning though are when $x$ is not a vector, but a function or some other structured object. 
(古典的なカーネル関数の例として$l_2$ノルム（ユークリッド距離）がある。$\left<\vec{x}, \vec{x}'\right>=\sqrt{\sum^D_i (x_i - x_i^{'})^2}$. 最も興味深いカーネル学習の応用のいくつかは$x$がベクトルではなく関数や他の構造化されたオブジェクトではあったりします。)
```{admonition} Primal & Dual Form　（主形式と双対形式）
:class: tip
**Dual Form** is what some call our model equation when it uses the kernels: $\hat{y} = \sum_i^N w_i \left<\vec{x}, \vec{x}_i\right>+ b$. To distinguish from the dual form, you can also refer to the usual model equation as the **Primal Form**  $\hat{y} = \vec{w}\vec{x} + b$. It also sounds cool. 
（**双対形式**とはカーネルを使うときに呼ばれるモデル式のことです。$\hat{y} = \sum_i^N w_i \left<\vec{x}, \vec{x}_i\right>+ b$. 双対形式と区別するために通常のモデル式を**主形式**と言うこともできます。$\hat{y} = \vec{w}\vec{x} + b$. 主形式もクールに聞こえますね。）
```

Kernel learning is a widely-used approach for learning potential energy functions and force fields in molecular modeling {cite}`scherer2020kernel,rupp2012fast`.
（カーネル学習は分子モデリングの際にポテンシャルエネルギー関数や力場を学習するための方法として広く使われています。{cite}`scherer2020kernel,rupp2012fast`）

## Solubility Example (溶解度の例)

Let's revisit the solubility AqSolDB{cite}`Sorkun2019` dataset from {doc}`regression`. Recall it has about 10,000 unique compounds with measured solubility in water (label) and 17 molecular descriptors (features).


それでは{doc}`regression`の溶解度データセットAqqSolDB{cite}`Sorkun2019`を再び見てみましょう。そのデータセットには水の溶解度の測定値(ラベル)と１７個の分子の記述子（特徴量）をもった約10,000件のユニークな化合物があるということを思い出してください。

## Running This Notebook (Notebookの実行)


Click the &nbsp;<i aria-label="Launch interactive content" class="fas fa-rocket"></i>&nbsp; above to launch this page as an interactive Google Colab. See details below on installing packages.
（上の&nbsp;<i aria-label="Launch interactive content" class="fas fa-rocket"></i>&nbsp;をクリックしてインタラクティブなGoogle Colabとしてこのページを開始しましょう。詳細は下記のインスト―リングパッケージを参照してください。）

````{tip（豆知識）} My title
:class: dropdown
To install packages, execute this code in a new cell. 
パッケージをインストールするには新しいセルでこのコードを実行してください。
```
!pip install dmol-book
```

If you find install problems, you can get the latest working versions of packages used in [this book here](https://github.com/whitead/dmol-book/blob/master/package/requirements.txt)
（もしインストールで問題があれば、[ここのノートブック](https://github.com/whitead/dmol-book/blob/master/package/requirements.txt)で使われている最新バージョンのパッケージを取得できます。）
````

As usual, the code below sets-up our imports.
(いつもと同じように、下記のコードは読み込みを行っています。)


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import jax.numpy as jnp
import jax
import dmol

In [ ]:
# soldata = pd.read_csv('https://dataverse.harvard.edu/api/access/datafile/3407241?format=original&gbrecs=true')
soldata = pd.read_csv(
    "https://github.com/whitead/dmol-book/raw/master/data/curated-solubility-dataset.csv"
)
features_start_at = list(soldata.columns).index("MolWt")
feature_names = soldata.columns[features_start_at:]
np.random.seed(0)

# standardize the features
soldata[feature_names] -= soldata[feature_names].mean()
soldata[feature_names] /= soldata[feature_names].std()

## Kernel Definition （カーネルの定義）

We'll start by creating our kernel function. *Our kernel function does not need to be differentiable*. In contrast to the functions we see in deep learning, we can use sophisticated and non-differentiable functions in kernel learning. For example, you could use a two-component molecular dynamics simulation to compute the kernel between two molecules.  We'll still implement our kernel functions in JAX for this example because it is efficient and consistent. Remember our kernel should take two feature vectors and return a scalar. In our example, we will simply use the $l_2$ norm. I will add one small twist though: dividing by the dimension. This makes our kernel output magnitude is independent of the number of dimensions of $x$. 
（カーネル関数を作るところから始めましょう。*カーネル関数は微分可能である必要はありません。*深層学習で見る関数とは対照的に、カーネル法では洗練された微分不可能な関数を使うことができます。例えば、２分子間のカーネルを計算するために２成分系の分子動力学シミュレーションを用いることができます。効率性と一貫性のため、この例でもJAXを用いてカーネル関数を実装していきます。カーネルは２つの特徴ベクトルを受け取ってスカラーを返すということを思い出してください。この例では、シンプルに$l_2$ノルムを使います。ただ、次元数で除算することでカーネルにひねりを少し与えてやります。こうすることでカーネル関数の出力のスケールは$x$の次元数に依存しなくなります。）

In [ ]:
def kernel(x1, x2):
    return jnp.sqrt(jnp.mean((x1 - x2) ** 2))

## Model Definition (モデル定義)


```{margin}
Since we're doing linear regression, you can compute the fit coefficients by just doing matrix algebra. We'll still approach this problem with gradient descent even though there are more efficient model-specific procedures.
（線形回帰を行っているので線形代数をするだけで回帰係数を計算することができます。他にもモデル特有の効率的な最適化手法が多くありますが、ここでは勾配降下法を用いてこの問題にアプローチしていくことにします。）
```

Now we define our regression model equation in the *dual form*. Remember that our function must always take the training data in to compute the distance to a new given point. We will use the batch feature of JAX to compute all the kernels simultaneously for our new point.
（ここでは、回帰モデル式を*双対形式*で定義します。任意の点との距離を計算するためにカーネル関数は常に訓練データを引数に持たなければならないということを思い出しましょう。新しい点に対して同時に全てのカーネルを計算するためにJAXのバッチ特徴量を使います。）


In [ ]:
def model(x, train_x, w, b):
    # make vectorized version of kernel
    vkernel = jax.vmap(kernel, in_axes=(None, 0), out_axes=0)
    # compute kernel with all training data
    s = vkernel(x, train_x)
    # dual form
    yhat = jnp.dot(s, w) + b
    return yhat


# make batched version that can handle multiple xs
batch_model = jax.vmap(model, in_axes=(0, None, None, None), out_axes=0)

## Training　(トレーニング)

We now have trainable weights and a model equation. To begin training, we need to define a loss function and compute its gradient. We'll use mean squared error as usual for the loss function. We can use regularization, as we saw previously, but will skip it for now. 
（この時点で訓練可能な重みとモデル式があります。トレーニングを始めるために、損失関数とその勾配を定義する必要があります。ここではいつも通りに平均二乗誤差を損失関数として用います。前回見たように、正則化を利用することもできますが今のところはスキップしましょう。）

In [ ]:
@jax.jit
def loss(w, b, train_x, x, y):
    return jnp.mean((batch_model(x, train_x, w, b) - y) ** 2)


loss_grad = jax.grad(loss, (0, 1))


# Get 80/20 split
N = len(soldata)
train = soldata[: int(N * 0.8)]
test = soldata[int(N * 0.8) :]

# convert from pandas dataframe to numpy arrays
train_x = train[feature_names].values
train_y = train["Solubility"].values
test_x = test[feature_names].values
test_y = test["Solubility"].values

We've defined our loss and split our data into training/testing. Now we will set-up the training parameters, including breaking up our training data into batches. An **epoch** is one iteration through the whole dataset. 
（損失関数を定義し、データを訓練データとテストデータに分けました。トレーニングデータをバッチに分けることも含めて、訓練パラメーターをここで設定していきます。**エポック**とはデータセット全体を通しての1回のイテレーションです。）

In [ ]:
eta = 1e-5
batch_size = 32
epochs = 10


# reshape into batches
batch_num = train_x.shape[0] // batch_size
# first truncate data so it's whole nubmer of batches
trunc = batch_num * batch_size
train_x = train_x[:trunc]
train_y = train_y[:trunc]
# split into batches
x_batches = train_x.reshape(-1, batch_size, train_x.shape[-1])
y_batches = train_y.reshape(-1, batch_size)


# make trainable parameters
# w = np.random.normal(scale = 1e-30, size=train_x.shape[0])
w = np.zeros(train_x.shape[0])
b = np.mean(train_y)  # just set to mean, since it's a good first guess

You may notice our learning rate, $\eta$, is unusually low at $10^{-5}$. It's because each training data point, for which we have about 8,000, contributes to the final $\hat{y}$. Thus if we take a large training step, it is can create very big changes to $\hat{y}$. 
（学習率$\eta$が$10^{-5}$と普通より小さいことに気が付いた人がいるかもしれません。この理由はというと、約8,000個あるトレーニングデータ点のそれぞれが最後の$\hat{y}$に寄与しているからです。それゆえ、もし学習ステップを大きくしてしまうと$\hat{y}$に大きな変化を及ぼしてしまう可能性があるのです。）

In [ ]:
loss_progress = []
test_loss_progress = []

for _ in range(epochs):
    # go in random order
    for i in np.random.randint(0, batch_num - 1, size=batch_num):
        # update step
        x = x_batches[i]
        y = y_batches[i]
        loss_progress.append(loss(w, b, train_x, x, y))
        test_loss_progress.append(loss(w, b, train_x, test_x, test_y))
        grad = loss_grad(w, b, train_x, x, y)
        w -= eta * grad[0]
        b -= eta * grad[1]
plt.plot(loss_progress, label="Training Loss")
plt.plot(test_loss_progress, label="Testing Loss")

plt.xlabel("Step")
plt.yscale("log")
plt.legend()
plt.ylabel("Loss")
plt.show()

One small change from previous training loops is that we randomized our batches in the `for` loop.
（先ほどのトレーニングループとの違いは`for`ループでバッチをランダム化したことです。）

In [ ]:
yhat = batch_model(test_x, train_x, w, b)
plt.plot(test_y, test_y, ":", linewidth=0.2)
plt.plot(test_y, yhat, "o")
plt.text(min(y) + 1, max(y) - 2, f"correlation = {np.corrcoef(test_y, yhat)[0,1]:.3f}")
plt.text(min(y) + 1, max(y) - 3, f"loss = {np.sqrt(np.mean((test_y - yhat)**2)):.3f}")
plt.title("Testing Data")
plt.show()

We can see our results show underfitting. As usual, I want to make this code execute fast so I have not done many epochs. You can increase the epoch number and watch the loss and correlation improve over time. 
（この結果は過小適合していると見受けられます。いつもと同じように、このコードを高速に実行させたいので多くのエポックを実行しませんでした。。エポック数を増やして損失と相関が時間経過とともに改善されていくところを見ることができます。）

## Regularization (正則化)


You'll notice that our trainable parameter number, by design, is equal to the number training data points. If we were to use a direct computation of the fit coefficients with a pseudo-inverse, we could run into problems because of this. Thus, most people add an additional regularization term to both make matrix algebra solutions tractable and because it seems wise with the large number of trainable parameters. Just like we saw in linear regression, L1 regression is known as **Lasso Ridge Regression** and L2 is known as **Kernel Ridge Regression**. Remember that L1 zeros-out specific parameters which was useful for interpreting the importance of features in linear regression. However, in the kernel setting this would only zero-out specific training data points and thus provides no real insight(usually, see {doc}`../dl/xai`). Kernel ridge regression is thus more popular in the kernel setting.
(読者の中には、訓練可能なパラメーター数が意図的に訓練データ点の個数と等しくしてあることに気づいた人がいるかもしれません。もし仮に擬似逆行列を用いて回帰係数を直接計算した場合、これのせいで問題に直面することがあります。それゆえ、行列計算の解が数学的に扱いやすくできるように、そして訓練可能パラメーターが多いほうが賢明だと思われるので正則化項を追加します。線形回帰モデルで見てきたように、L1回帰は**ラッソ・リッジ回帰**として知られ、L2回帰は**カーネルリッジ回帰**として知られます。L1の0でない特定のパラメーターが線形回帰において特徴量の重要度を解釈するのに便利であることを思い出してください。しかしながら、カーネル法では0でない特定の訓練データ点が得られるだけで何も現実的な洞察を与えてはくれません({doc}`../dl/xai`)参照)。そういうわけでカーネルリッジ回帰はカーネル法の範疇では有名です。

## Training Curves (学習曲線)

The bias-variance trade-off from {doc}`../ml/regression` showed how increasing model complexity could reduce model bias (more expressive and able to fit data better) at the cost of increased model variance (more sensitive to training data choice and amount). The model complexity was controlled by adjusting feature number. In kernel learning, we cannot control feature number because it is always equal to the number of training data points. Thus, we can only control hyperparameters like the choice of kernel, regularization, learning rate, etc. To assess these effects, we usually do not only compute test loss because that is highly-connected to the amount of training data you have. More training data means more sophisticated models and thus lower loss. So it is common in kernel learning especially to show how the test-loss changes a function of training data amount. These are presented as log-log plots due to the large magnitude changes in these. These are called **training curves** (or sometimes **learning curves**). Training curves can be applied broadly in ML and deep learning, but you'll most often see them in kernel learning.
（{doc}`../ml/regression`のバイアス-バリアンストレードオフは、モデルの複雑さを増加させると、モデルのバリアンス（トレーニングデータの選択の仕方と量により強く影響を受ける）の増加を犠牲にしてどれほどモデルのバイアス（より表現力が高く、より良くデータに適合できる）を軽減できるかについて示しました。モデルの複雑さは特徴量の個数を調整することによって制御することができました。カーネル法では特徴量の個数をコントロールしません。というのもそれが訓練データ点の個数と常に一致するからです。それゆえ、カーネルの選択や正則化、学習率などのようなハイパーパラメーターをコントロールするだけになります。このハイパーパラメーターの効果を評価するためにテストの損失を計算するだけでは終わりません。なぜならその損失がトレーニングデータの量に強く関係しているからです。多くの訓練データがあるということはモデルがより洗練されるということを意味し損失の値が減少します。なので、とりわけカーネル学習ではトレーニングデータの量に対してテスト損失の値がどれほど変わるのかを示すということは常識的なことです。値が大きなスケールで変化するのでこれらは対数-対数プロットで示されます。これらは**学習曲線**(もしくは**Learning curves**)と呼ばれています。学習曲線は機械学習や深層学習で広く使われていますが、カーネル学習でも頻繁に見ることになります。）
Let's revisit our solubility model and compare L1 and L2 regularization with a training curve. Note that this code is very slow because we must compute $M$ models, where $M$ is the number of points we want on our training curve. To keep things efficient for this textbook, I'll use few points on the curve.
（溶解度のモデルに再び戻って学習曲線を用いてL1とL2の正則化を比較してみましょう。注意しておきますが、$M$個のモデル（$M$は学習曲線で取得したい点の個数）を計算しなければならないので、このコードは非常に遅くなります。このテキストでは効率性を保つために曲線上の点として数個を用いることにします。）
First, we'll turn our training procedure into a function.
（最初にトレーニングの過程を関数にまとめます。）

In [ ]:
def fit_model(loss, npoints, eta=1e-6, batch_size=16, epochs=25):

    sample_idx = np.random.choice(
        np.arange(train_x.shape[0]), replace=False, size=npoints
    )
    sample_x = train_x[sample_idx, :]
    sample_y = train_y[sample_idx]

    # reshape into batches
    batch_num = npoints // batch_size
    # first truncate data so it's whole nubmer of batches
    trunc = batch_num * batch_size
    sample_x = sample_x[:trunc]
    sample_y = sample_y[:trunc]
    # split into batches
    x_batches = sample_x.reshape(-1, batch_size, sample_x.shape[-1])
    y_batches = sample_y.reshape(-1, batch_size)

    # get loss grad
    loss_grad = jax.grad(loss, (0, 1))

    # make trainable parameters
    # w = np.random.normal(scale = 1e-30, size=train_x.shape[0])
    w = np.zeros(sample_x.shape[0])
    b = np.mean(sample_y)  # just set to mean, since it's a good first guess
    for _ in range(epochs):
        # go in random order
        for i in np.random.randint(0, batch_num - 1, size=batch_num):
            # update step
            x = x_batches[i]
            y = y_batches[i]
            grad = loss_grad(w, b, sample_x, x, y)
            w -= eta * grad[0]
            b -= eta * grad[1]
    return loss(w, b, sample_x, test_x, test_y)


# test it out
fit_model(loss, 256)

Now we'll create L1 and L2 version of our loss. We must choose the *strength* of the regularization. Since our weights are less than 1, I'll choose much stronger regularization for the L2. These are hyperparameters though and you can adjust them to improve your fit. 
(ここでは私たち用のL1とL2バージョンの損失関数を作っていきます。正則化の*強度(strength)*を選択しなければなりません。重みは1より小さいのでL2に対してはよりもっと強い正則化の強度を選択することにします。これらはハイパーパラメーターですがフィッティングを向上させるために調整しても問題ないです。)

In [ ]:
@jax.jit
def loss_l1(w, b, train_x, x, y):
    return jnp.mean((batch_model(x, train_x, w, b) - y) ** 2) + 1e-2 * jnp.sum(
        jnp.abs(w)
    )


@jax.jit
def loss_l2(w, b, train_x, x, y):
    return jnp.mean((batch_model(x, train_x, w, b) - y) ** 2) + 1e2 * jnp.sum(w**2)

And now we can generate the points necessary for our curves!
（そして、学習曲線に必要な点を生成します！）

In [ ]:
nvalues = [32, 256, 512, 1024, 2048, 1024 * 5]

nor_losses = [fit_model(loss, n) for n in nvalues]
l1_losses = [fit_model(loss_l1, n) for n in nvalues]
l2_losses = [fit_model(loss_l2, n) for n in nvalues]

In [ ]:
plt.plot(nvalues, nor_losses, label="No Regularization")
plt.plot(nvalues, l1_losses, label="L1 Regularization")
plt.plot(nvalues, l2_losses, label="L2 Regularization")
plt.legend()
plt.xlabel("Training Data Amount")
plt.ylabel("Test Loss")
plt.gca().set_yscale("log")
plt.gca().set_xscale("log")

Finally, we see our training curves showing the different approaches. Regularization has some effect on final loss on the test data. It is hard to say if L1 and L2 are simply worse, or if I need to tune the regularization strength more. Nevertheless, this plot shows you how we typically evaluated kernel learning methods. 
(最後に別々のアプローチをとった学習曲線を見てみましょう。正則化はテストデータの最終的な損失になにかしらの影響を及ぼしています。L1とL2正則化は単純に悪くなっているかの是非や正則化の強度をもっと調整したほうが良いかどうかについて言及することは困難です。とは言え、このプロットを見る限りでは我々がカーネル学習法を典型的に評価したということはわかります。)

## Exercises (練習問題)

1. Compute the analytical gradient for the dual form regression equation and use it to describe why the kernel function does not need to be differentiable.（双対形式の回帰式から勾配を計算し、その計算結果を用いてカーネル関数がなぜ微分可能である必要がないのかについて説明しなさい。）
2. Is it faster or slower to do training with kernel learning? Explain（カーネル学習でトレーニングを行うのは速いか遅いか？説明しなさい。）
3. Is it faster or slower to do inference with kernel learning? Explain（カーネル学習で予測するのは速いか遅いか？説明しなさい。）
4. How can we modify Equation 4.2 to do classification? （分類タスクを行うために式4.2をどのように修正すればよいか？）
5. Do the weight values give relative importance of training examples regardless of kernel?（カーネルの有無に関わらず、重みの値がトレーニングデータの相対的な重要度を示しているか？）
6. Create a training curve from the above example showing 5 different L1 regularization strengths. Why might regularization not matter here?（上記の例からL1正則化の強度が異なる5つの学習曲線を作成しなさい。この例で正則化が意味をなさないのないのはなぜか？）

## Chapter Summary(章のまとめ)

* In this section we introduced kernel learning, which is a method to transform features into distance between samples.（この章ではカーネル学習を紹介した。カーネル学習は特徴量をサンプル間の距離に変換する方法である。）
* A kernel function takes two arguments and outputs a scalar. A kernel function must have three properties: positive, symmetric, and point-separating.（カーネル関数は2つの引数を受け取ってスカラーを出力する。カーネル関数は3つの性質を持たなければならない（正定性(positive)・対称性(symmetry)・Point-separating）。）
* The distance function (inner product) is a kernel function.（距離関数（内積）はカーネル関数である。）
* Kernel functions do not need to be differentiable.（カーネル関数は微分可能である必要はない。）
* Kernel learning is appropriate when you would like to use features that can only be specified as a binary kernel function.（カーネル学習は二変数カーネル関数だけで定められる特徴量を使いたいときに相応しい手法である。）
* The number of trainable parameters in a kernel model is proportional to number of training points, not dimension of features.（カーネルモデルの訓練可能パラメーターの個数は特徴量の次元数ではなく訓練データ点の個数に比例する。）

## Cited References

```{bibliography}
:style: unsrtalpha
:filter: docname in docnames
```